## Пример классификации цифр в PyTorch

In [1]:
import torch
import torchvision

import torch.nn as nn

In [2]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=torchvision.transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=torchvision.transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=100, 
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw



In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title("Ground Truth: {}".format(example_targets[i]))
    plt.xticks([])
    plt.yticks([])

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        
    def forward(self, inputs):
        output_1 = self.relu(self.layer_1(inputs))
        output = self.layer_2(output_1)
        
        return output

In [ ]:
model = NeuralNet(28*28, 500, 10)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
total_step = len(train_loader)
for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, 28*28)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 5, i+1, total_step, loss.item()))

In [1]:
# На этапе предсказания не требуется вычислять градиенты:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

NameError: name 'model' is not defined

<div class="alert alert-info">

<h3> Задание (выполнять в отдельном файле)</h3>
<p></p>
Реализовать в PyTorch произвольную <b>полносвязную</b> архитектуру для решения задачи бинарной классификации: снаружи или внутри помещения сделана фотография. Для загрузки изображений используйте следующий код:
 <p></p>
</div>

In [1]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

import os
import zipfile

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# data_folder = r'/content/drive/My Drive/data/'

# !unzip '/content/drive/My Drive/data/cian_dataset.zip' -d '/content/drive/My Drive/data/'

In [3]:
if torch.cuda.is_available():
    dev = 'cuda:0'
else:  
    dev = 'cpu'
device = torch.device(dev) 

In [47]:
transform = transforms.Compose([transforms.Resize((25,25)), transforms.ToTensor()])

train_data = torchvision.datasets.ImageFolder('C:/Users/Lenovo/Documents/train/',
                                              transform=transform)

trainset = torch.utils.data.Subset(train_data, list(range(0, len(train_data), 6)))

In [48]:
len(trainset)

9221

In [49]:
train_set, val_set = torch.utils.data.random_split(trainset, [6000, 3221])
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=256,
                                                shuffle=True, num_workers=2)
val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=256,
                                              shuffle=True, num_workers=2)

In [50]:
class CustomNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomNN, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size)
        self.layer_2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        output_1 = self.relu(self.layer_1(inputs))
        output = self.layer_2(output_1)
        return output

In [51]:
model = CustomNN(25*25*3,10,1).to(device)
print(model)

CustomNN(
  (layer_1): Linear(in_features=1875, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
)


In [52]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [53]:
  # обучение
torch.manual_seed(10)

epochs = 5

total_step = len(train_data_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.to(device)
        images = images.reshape(-1, 25*25*3)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
               .format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [1/24], Loss: 0.7133
Epoch [1/5], Step [2/24], Loss: 0.7094
Epoch [1/5], Step [3/24], Loss: 0.6855
Epoch [1/5], Step [4/24], Loss: 0.6877
Epoch [1/5], Step [5/24], Loss: 0.6977
Epoch [1/5], Step [6/24], Loss: 0.6752
Epoch [1/5], Step [7/24], Loss: 0.6651
Epoch [1/5], Step [8/24], Loss: 0.6667
Epoch [1/5], Step [9/24], Loss: 0.6396
Epoch [1/5], Step [10/24], Loss: 0.6476
Epoch [1/5], Step [11/24], Loss: 0.6539
Epoch [1/5], Step [12/24], Loss: 0.6464
Epoch [1/5], Step [13/24], Loss: 0.6390
Epoch [1/5], Step [14/24], Loss: 0.6309
Epoch [1/5], Step [15/24], Loss: 0.6430
Epoch [1/5], Step [16/24], Loss: 0.6447
Epoch [1/5], Step [17/24], Loss: 0.6590
Epoch [1/5], Step [18/24], Loss: 0.6263
Epoch [1/5], Step [19/24], Loss: 0.6042
Epoch [1/5], Step [20/24], Loss: 0.5963
Epoch [1/5], Step [21/24], Loss: 0.6197
Epoch [1/5], Step [22/24], Loss: 0.6251
Epoch [1/5], Step [23/24], Loss: 0.5886
Epoch [1/5], Step [24/24], Loss: 0.5724
Epoch [2/5], Step [1/24], Loss: 0.5900
Epoch [2/5

In [54]:
# оценка качества
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_data_loader:
        images = images.to(device)
        images = images.reshape(-1, 25*25*3)
        labels = labels.reshape(len(labels), 1)
        labels = labels.type(torch.FloatTensor)
        labels = labels.to(device)
        
        outputs = model(images)
        predicted = (torch.sigmoid(outputs.data) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy: {} %'.format(100 * correct / total))

Accuracy: 83.4523439925489 %
